# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [19]:
import pandas as pd

# Load the data
jeopardy_df = pd.read_csv('jeopardy.csv')

# Clean column names
jeopardy_df.columns = jeopardy_df.columns.str.strip()

pd.set_option('display.max_colwidth', None)

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [21]:
def filter_questions_by_words(dataframe, words):
    """
    Filter the DataFrame for questions containing all the specified words.
    
    Args:
    dataframe (pd.DataFrame): The Jeopardy DataFrame.
    words (list of str): Words to look for in each question.
    
    Returns:
    pd.DataFrame: Filtered DataFrame with questions containing all the specified words.
    """
    filter_condition = dataframe['Question'].apply(lambda x: all(word.lower() in x.lower() for word in words))
    return dataframe[filter_condition]

# Test the function with words ["King", "England"]
filtered_questions = filter_questions_by_words(jeopardy_df, ["King", "England"])
print(filtered_questions['Question'])

4953                    Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
6337      In retaliation for Viking raids, this "Unready" king of England attacks Norse areas of the Isle of Man
9191                    This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt
11710               This Scotsman, the first Stuart king of England, was called "The Wisest Fool in Christendom"
13454                                       It's the number that followed the last king of England named William
                                                           ...                                                  
208295        In 1066 this great-great grandson of Rollo made what some call the last Viking invasion of England
208742                      Dutch-born king who ruled England jointly with Mary II & is a tasty New Zealand fish
213870                In 1781 William Herschel discovered Uranus & initially named it after this

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [27]:
import re

def filter_questions_by_words_more_robust(dataframe, words):
    """
    Further enhanced filter for the DataFrame to ensure case insensitivity, prevent substring matches,
    and handle possessive cases.
    
    Args:
    dataframe (pd.DataFrame): The Jeopardy DataFrame.
    words (list of str): Words to look for in each question.
    
    Returns:
    pd.DataFrame: Filtered DataFrame with questions containing all the specified words, including possessive cases.
    """
    # Use regex to find whole words, accounting for case insensitivity and possessive cases
    word_regex = lambda w: r'\b' + re.escape(w) + r'(\'s)?\b'
    filter_condition = dataframe['Question'].apply(
        lambda x: all(re.search(word_regex(word), x, flags=re.IGNORECASE) for word in words)
    )
    return dataframe[filter_condition]

# Example usage
test_words = ["King", "England"]
filtered_questions = filter_questions_by_words_more_robust(jeopardy_df, test_words)
print(filtered_questions['Question'])

4953                    Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
6337      In retaliation for Viking raids, this "Unready" king of England attacks Norse areas of the Isle of Man
9191                    This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt
11710               This Scotsman, the first Stuart king of England, was called "The Wisest Fool in Christendom"
13454                                       It's the number that followed the last king of England named William
                                                           ...                                                  
201168                                  Popular Saint-Exupery character waiting around to become king of England
204778                                 This southern colony was named for a king of England, the II of that name
208742                      Dutch-born king who ruled England jointly with Mary II & is a tasty 

4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   While most of the values in the `" Value"` column represent a dollar amount as a string, note that some do not &mdash; these values will need to be handled differently!

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [29]:
def convert_value_to_float(value):
    try:
        # Remove the dollar sign and commas, then convert to float
        return float(value.replace('$', '').replace(',', ''))
    except ValueError:
        # Return 0 for any values that can't be converted to floats
        return 0

# Apply the conversion function to the "Value" column and create a new column for the float values
jeopardy_df['FloatValue'] = jeopardy_df['Value'].apply(convert_value_to_float)

# Function to calculate the average difficulty for questions containing a specific word
def filter_questions_by_word_and_calculate_difficulty(dataframe, word):
    """
    Filter questions containing the word and calculate the average difficulty (value) of those questions.
    
    Args:
    dataframe (pd.DataFrame): The Jeopardy DataFrame.
    word (str): Word to look for in each question.
    
    Returns:
    float: Average value of filtered questions.
    """
    filtered_df = dataframe[dataframe['Question'].str.contains(r'\b' + re.escape(word) + r'\b', case=False, regex=True)]
    return filtered_df['FloatValue'].mean()

# Calculate the average difficulty for questions containing the word "King"
average_difficulty_king = filter_questions_by_word_and_calculate_difficulty(jeopardy_df, "King")
average_difficulty_king

803.2981049562683

5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [31]:
def unique_answers_count(dataframe, word):
    """
    Count unique answers for questions containing the specified word.

    Args:
    dataframe (pd.DataFrame): The Jeopardy DataFrame.
    word (str): Word to look for in each question.

    Returns:
    pd.Series: Counts of unique answers sorted in descending order.
    """
    # Filter dataframe for questions containing the specified word
    filtered_df = dataframe[dataframe['Question'].str.contains(r'\b' + re.escape(word) + r'\b', case=False, regex=True)]
    # Count unique answers and sort them
    answer_counts = filtered_df['Answer'].value_counts()
    return answer_counts

# Example usage: Calculate the unique answer counts for questions containing the word "King"
unique_answers_king = unique_answers_count(jeopardy_df, "King")
print(unique_answers_king)

Answer
Henry VIII                           54
Solomon                              31
Louis XIV                            31
David                                29
Sweden                               24
                                     ..
Oscar                                 1
Henderson                             1
(Catherine) Howard                    1
"The Circle Of Life"                  1
a pyramid (the pyramids accepted)     1
Name: count, Length: 1499, dtype: int64


6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [33]:
def keyword_trend_by_decade(dataframe, keyword):
    """
    Analyze the frequency of a keyword in questions across different decades.
    
    Args:
    dataframe (pd.DataFrame): The Jeopardy DataFrame.
    keyword (str): The keyword to search for in questions.
    
    Returns:
    pd.DataFrame: Counts of keyword appearance by decade.
    """
    # Extract year from 'Air Date' and create a new column for the decade
    dataframe['Decade'] = dataframe['Air Date'].str[:4].astype(int) // 10 * 10
    # Filter questions containing the keyword
    keyword_filter = dataframe['Question'].str.contains(keyword, case=False, regex=False)
    # Count occurrences by decade
    return dataframe[keyword_filter].groupby('Decade').size()

# Example usage
computer_trends = keyword_trend_by_decade(jeopardy_df, "Computer")
print(computer_trends)

Decade
1980      6
1990     98
2000    268
2010     59
dtype: int64


## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.